# Unit 5 - Financial Planning

In [2]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import warnings

warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)


%matplotlib inline

In [3]:
# Load .env enviroment variables
load_dotenv()

True

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [6]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3
#my_btc = float(my_btc)
#my_eth = float(my_eth)

In [7]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

In [8]:
# Fetch current BTC price
my_btc = requests.get(btc_url).json()
btc_value = my_btc['data']['1']['quotes']['USD']['price']
# Fetch current ETH price
my_eth = requests.get(eth_url).json()
eth_value = my_eth['data']['1027']['quotes']['USD']['price']
# Compute current value of my crpto
my_btc_value = 1.2 * btc_value
my_eth_value = 5.3 * eth_value 
portfolio_value= btc_value + eth_value
# Print current crypto wallet balance
print(f"The current value of your {'my btc'} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {'my eth'} ETH is ${my_eth_value:0.2f}")

ConnectTimeout: HTTPSConnectionPool(host='api.alternative.me', port=443): Max retries exceeded with url: /v2/ticker/Ethereum/?convert=USD (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000015F710E0450>, 'Connection to api.alternative.me timed out. (connect timeout=None)'))

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
api_key= os.getenv("ALPACA_API_KEY")
api_secret= os.getenv("ALPACA_SECRET_KEY")
#alpaca_endpoint = os.getenv("ALPACA_ENDPOINT")
api = tradeapi.REST(api_key, api_secret, api_version="v2")
#base_url = alpaca_endpoint

In [ ]:
# Format current date as ISO format
start_date = pd.Timestamp("2024-04-19", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2024-04-19", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
stockticker= api.get_bars(tickers, timeframe, start= start_date, end= end_date).df
# Separate ticker data
AGG = stockticker[stockticker['symbol']=='AGG'].drop(columns='symbol', axis=1)
SPY = stockticker[stockticker['symbol']=='SPY'].drop(columns='symbol', axis=1)

# Concatenate the ticker DataFrames
#concatenated_df = pd.concat([agg_df, agg_df])
portfolio_concat_df = pd.concat([AGG, SPY], axis=1, keys=['AGG', 'SPY'])

# Preview DataFrame
portfolio_concat_df

In [ ]:
# Pick AGG and SPY close prices
agg_close_price = portfolio_concat_df['AGG']['close'].values[0]
spy_close_price = portfolio_concat_df['SPY']['close'].values[0]

print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
# YOUR CODE HERE!
my_agg_value = agg_close_price * my_agg
my_spy_value = spy_close_price * my_spy
# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!
monthly_income= 12000
# Consolidate financial assets data
# YOUR CODE HERE!
total_crpto_value= my_btc_value + my_eth_value
total_shares_value= my_agg_value + my_spy_value
savings_data = [total_crpto_value, total_shares_value]
# Create savings DataFrame
# YOUR CODE HERE!
savings_df= pd.DataFrame(savings_data, columns=['amount'], index=['crypto', 'stock/bond'])
# Display savings DataFrame
display(savings_df)

In [ ]:
# Plot savings pie chart
savings_df.plot.pie(y='amount', title='Composition of Personal Savings', autopct='%1.1f%%')


In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!
total_portfolio_value = savings_df['amount'].sum()

# Validate saving health
# YOUR CODE HERE!
if total_portfolio_value > emergency_fund:
    print("Congratulations! You have enough money in this fund.")
elif total_portfolio_value == emergency_fund:
    print("Congratulations on reaching this important financial goal.")
else:
    shortfall = emergency_fund - total_portfolio_value
    print(f"You are ${shortfall} away from reaching your emergency fund")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2019-04-19', tz='America/New_York').isoformat()
end_date = pd.Timestamp("2024-04-19", tz="America/New_York").isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
five_yr_df = api.get_bars(tickers, timeframe, start=start_date, end=end_date).df
# Reorganize the DataFrame
# Separate ticker data
fiveyr_spy_df = five_yr_df[five_yr_df['symbol']=='SPY'].drop(columns='symbol')
fiveyr_agg_df = five_yr_df[five_yr_df['symbol']=='AGG'].drop(columns='symbol')
#axis=1

# Concatenate the ticker DataFrames
fiveyr_stock_data = pd.concat([fiveyr_spy_df, fiveyr_agg_df], axis=1, keys=['SPY', 'AGG'])

# Display sample data
fiveyr_stock_data.head()


In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!
mc_sim = MCSimulation(
    portfolio_data = fiveyr_stock_data,
    weights = [0.6, 0.4],
    num_simulation = 500,
    num_trading_days = 252*30)



In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!
mc_sim.portfolio_data.head()


In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!
mc_sim.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!
mc_sim.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!
mc_sim.plot_distribution()


### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary_stats = mc_sim.summarize_cumulative_return()

# Print summary statistics
print(summary_stats)

# Extract the lower and upper bounds of the 95% confidence interval
ci_lower = summary_stats.loc["95% CI Lower"]
ci_upper = summary_stats.loc["95% CI Upper"]

# Print the lower and upper bounds of the 95% confidence interval
print(f"Lower Bound of the 95% Confidence Interval: ${ci_lower}")
print(f"Upper Bound of the 95% Confidence Interval: ${ci_upper}"

      this question always gave me an error no matter hat i did or formatted it

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")